# Export variables for validation
- This script is used to export variables from simulations and observations;
- Simulations: CNTL, TRAF at FR-Capitole. 

In [1]:
import os
import xarray as xr
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cftime
import string
import matplotlib.dates as mdates
home_path = '/gws/nopw/j04/duicv/yuansun/'

In [2]:
archive = f'{home_path}0_urban_traffic/archive'
psites = ["FR-Capitole"]
msites = ["FR-Cap"]
START = ['1994']
END = ['2005']
var0 = ['LWdown', 'SWdown']
var1 = ['LWdown', 'SWdown','LWup', 'SWup', 'Qh', 'Qle', 'Qtau']
var2 = ['FIRE_U', 'FSR', 'FSH_U', 'EFLX_LH_TOT_U', 'TAUX']
zone = [1]
start_date = ['2004-02-20T00:30:00']
end_date = ['2005-02-28T22:30:00']

In [4]:
for sitesequence in range(len(psites)):
    GRIDNAME = psites[sitesequence]
    GRIDNAME_single = msites[sitesequence]
    START_year = START[sitesequence]
    time_zone = zone[sitesequence] 
    start_time = start_date[sitesequence]
    end_time = end_date[sitesequence]
    
    datm = f'{home_path}0_lcz_sp/UrbanPlumber/datm_files/'+ GRIDNAME_single + '/CLM1PT_data/CTSM_DATM_' + GRIDNAME_single + '_' + START[sitesequence] + '-' + END[sitesequence] + '.nc'
    ds_datm = xr.open_dataset(datm)
    
    observation = f'{home_path}0_lcz_sp/UrbanPlumber/Urban-PLUMBER_FullCollection_v1/' + GRIDNAME + '/timeseries/'+ GRIDNAME + '_clean_observations_v1.nc'
    ds_ob = xr.open_dataset(observation)
    
    cntl = f'{archive}/{GRIDNAME_single}_cntl/lnd/hist/{GRIDNAME_single}_cntl.clm2.h0.2004-02-20-03600.nc' 
    ds_cntl = xr.open_dataset(cntl)
    traf = f'{archive}/{GRIDNAME_single}_traffic/lnd/hist/{GRIDNAME_single}_traffic.clm2.h0.2004-02-20-03600.nc'
    ds_traf = xr.open_dataset(traf)
    
    ds_cntl['time']=ds_cntl['time'].dt.round('min')
    ds_cntl['time']=ds_cntl['time'].dt.ceil('min')
    ds_traf['time']=ds_traf['time'].dt.round('min')
    ds_traf['time']=ds_traf['time'].dt.ceil('min')
    
    sliced_ds_datm = ds_datm.sel(time=slice(start_time, end_time))[var0]
    df_datm = sliced_ds_datm.to_dataframe().reset_index()
    
    #sliced_ds_ko = ds_ko.sel(time=slice(start_time, end_time))[var1]
    sliced_ds_cntl = ds_cntl.sel(time=slice(start_time, end_time))[var2]
    df_cntl = sliced_ds_cntl.to_dataframe().reset_index()
    df_cntl.drop(columns=['lndgrid'], inplace=True)
    df_cntl = df_cntl.rename(columns={'FIRE_U': 'LWup_cntl', 'FSR': 'SWup_cntl','FSH_U': 'Qh_cntl', 'EFLX_LH_TOT_U': 'Qle_cntl', 'TAUX': 'Qtau_cntl'})
    df_cntl['Qtau_cntl'] = - df_cntl['Qtau_cntl']
    df_cntl['Rn_cntl'] = df_datm['LWdown'] + df_datm['SWdown'] - df_cntl['SWup_cntl'] - df_cntl['LWup_cntl']
    
    sliced_ds_traf = ds_traf.sel(time=slice(start_time, end_time))[var2]
    df_traf = sliced_ds_traf.to_dataframe().reset_index()
    df_traf.drop(columns=['lndgrid'], inplace=True)
    df_traf = df_traf.rename(columns={'FIRE_U': 'LWup_traf', 'FSR': 'SWup_traf','FSH_U': 'Qh_traf', 'EFLX_LH_TOT_U': 'Qle_traf', 'TAUX': 'Qtau_traf'})
    df_traf['Qtau_traf'] = - df_traf['Qtau_traf']
    df_traf['Rn_traf'] = df_datm['LWdown'] + df_datm['SWdown'] - df_traf['SWup_traf'] - df_traf['LWup_traf']
     
    sliced_ds_ob = ds_ob.sel(time=slice(start_time, end_time))[var1]
    df_ob = sliced_ds_ob.to_dataframe().reset_index()
    df_ob = df_ob.rename(columns={'LWdown': 'LWdown_obs', 'SWdown': 'SWdown_obs', 'LWup': 'LWup_obs', 
                                  'SWup': 'SWup_obs', 'Qh': 'Qh_obs', 'Qle': 'Qle_obs', 'Qtau': 'Qtau_obs'})
    df_ob['Rn_obs'] = df_ob['LWdown_obs'] + df_ob['SWdown_obs']-df_ob['SWup_obs']-df_ob['LWup_obs']
    df = pd.merge(df_ob, df_cntl, on='time').merge(df_traf, on='time')
    df.loc[df['LWup_obs'].isna(), ['LWup_cntl', 'LWup_traf', 'Rn_obs', 'Rn_cntl', 'Rn_traf']] = np.nan
    df.loc[df['SWup_obs'].isna(), ['SWup_cntl', 'SWup_traf', 'Rn_obs', 'Rn_cntl', 'Rn_traf']] = np.nan
    df.loc[df['Qle_obs'].isna(), ['Qle_cntl', 'Qle_traf']] = np.nan
    df.loc[df['Qh_obs'].isna(), ['Qh_cntl', 'Qh_traf']] = np.nan
    df.loc[df['Qtau_obs'].isna(), ['Qtau_cntl', 'Qtau_traf']] = np.nan
    
    df['hour']=(df.time.dt.hour + time_zone) % 24
    df['minute']=df.time.dt.minute
    df['diurnal']=df['hour'].astype(str).str.zfill(2) + ':' + df['minute'].astype(str).str.zfill(2)
    df.to_csv( f'./data_for_figure/{GRIDNAME}.csv', index=False)

In [5]:
ds_traf['URBAN_HEAT'].max()

<xarray.DataArray 'URBAN_HEAT' ()> Size: 4B
array(38.375435, dtype=float32)

In [6]:
ds_traf['TRAFFICFLUX'].max()

<xarray.DataArray 'TRAFFICFLUX' ()> Size: 4B
array(57.446, dtype=float32)

In [51]:
ds_traf['VEHICLE_FLOW'][0:1,0]

<xarray.DataArray 'VEHICLE_FLOW' (time: 1)> Size: 4B
array([7046.4], dtype=float32)
Coordinates:
  * time     (time) datetime64[ns] 8B 2004-02-20T01:00:00
Attributes:
    long_name:      vehicle flow
    units:          vehicle per hour
    cell_methods:   time: instantaneous
    landunit_mask:  unknown

In [53]:
ds_traf['VEHICLE_SPEED'][:1,0].values

array([11.1], dtype=float32)

In [73]:
ds_traf['TRAFFICFLUX'][0,0].values

array(0.00099493, dtype=float32)